# Image Classification with Multi-Layer Perceptron (MLP)

In this Jupyter notebook, we will implement a two-layer multi-layer perceptron (MLP) with ReLU activation function using PyTorch for image classification on the MNIST dataset. The main objective of this project is to achieve a higher accuracy on the MNIST classification task than the previous implementation of a logistic regression model using PyTorch.

In [8]:
# import our libraries
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

In [9]:
# import our datasets
mnist_train = datasets.MNIST(root='./datasets', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root='./datasets', train=False, transform=transforms.ToTensor(), download=True)

# use dataloader to put our in batches and shuffle
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=200, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=200, shuffle=False)

We will first build our model from scratch using PyTorch and numpy functions then we will take the object oriented aproach and build our model by using a python class that inherits fron the nn.Module class

In [10]:
# building and training our model from scratch
epochs = 100
lr = 0.001

# intantiate our weights and biases for the two layers
# first layer parameters
W1 = torch.randn(784,512)/np.sqrt(782)
W1.requires_grad_()
b1 = torch.zeros(512, requires_grad=True)
# second layer parameters
W2 = torch.randn(512,10)/np.sqrt(500)
W2.requires_grad_()
b2 = torch.zeros(10, requires_grad=True)

# optimizer and loss function
optimizer = torch.optim.Adam(params=[W1,b1,W2,b2], lr=lr)
criteria = nn.CrossEntropyLoss()

# iterate through our training data 
for epoch in tqdm(range(epochs)):
  for image, label in train_loader:
    # zero out the gradient
    optimizer.zero_grad()
    # forward pass
    x = image.view(-1,784)
    y1_lin = torch.matmul(x,W1) + b1
    y1 = F.relu(y1_lin) #pass through the activation function
    y2 = torch.matmul(y1,W2) + b2
    loss = criteria(y2, label)
    # backward pass
    loss.backward()
    optimizer.step()

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
# testing our module
correct = 0
total = len(mnist_test)

# iterate through the test set
with torch.no_grad():
  for image, label in tqdm(test_loader):
    x = image.view(-1,784)
    hidden = torch.matmul(x,W1) + b1
    hidden = F.relu(hidden)
    output = torch.matmul(hidden, W2) + b2
        
    prediction = torch.argmax(output,dim=1)
    correct += torch.sum((prediction == label).float())
print('Test accuracy: {}'.format(correct/total))  

  0%|          | 0/50 [00:00<?, ?it/s]

Test accuracy: 0.9832000136375427


We got a 98% accuracy in image classification on our test set, we can see that our MLP performed better than our simple logistic regression module.
We will now refactor our code and take an object oriented approach to developing our two layer module.

In [18]:
# define our module
class MultiLayerModule(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(784,512)
    self.layer2 = nn.Linear(512,10)

  def forward(self, x):
    # first layer
    x = x.view(-1,784) #flatten input tensor
    x = self.layer1(x)
    x = F.relu(x)
    # second layer
    y = self.layer2(x)
    return y

# create a module object   
model = MultiLayerModule()
lr = 0.001
epochs = 100
# define our optimizer and loss function
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
criteria = nn.CrossEntropyLoss()

#iterate through our training data
for epoch in tqdm(range(epochs)):
  for image, label in train_loader:
    # zero out gradient
    optimizer.zero_grad()
    # forward pass
    x = image
    y = model.forward(x)
    loss = criteria(y, label)
    # backwards pass
    loss.backward()
    optimizer.step()

# testing our module
correct = 0
total = len(mnist_test)
with torch.no_grad():
  for image, label in tqdm(test_loader):
    x = image
    y = model.forward(x)

    prediction = torch.argmax(y, dim=1)
    correct += torch.sum((prediction == label).float())
print(f'Test Accuracy: {correct/total}')

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Test Accuracy: 0.9829999804496765


##Conclusion
In conclusion, we have successfully built and trained a two-layer multilayer perceptron model with a ReLU activation function using PyTorch for image classification on the MNIST dataset. We achieved an impressive 98% accuracy on the test data, demonstrating the effectiveness of our model. Our results suggest that a simple MLP with only two layers can perform well on the MNIST dataset, which is a well-known benchmark for image classification tasks. Overall, this notebook provides a solid foundation for building and training deep learning models using PyTorch, and can be used as a starting point for further exploration of more complex models and datasets.